# DanteGen: a tercet generator

The purpose of this project is to create a generative model based on Dante's Divine Comdey loosely based on what A. Karpathy [did](http://karpathy.github.io/2015/05/21/rnn-effectiveness/) with Shakespeare's works.
I'm going to follow step by step what J. Howard [did](https://github.com/fastai/courses/blob/master/deeplearning1/nbs/char-rnn.ipynb) in one of his great lessons on fast.ai.

## Getting the data

In [1]:
import numpy as np
from keras.models import Sequential, Model
from keras.layers import Input, Embedding, Reshape, merge, LSTM, Bidirectional
from keras.layers import TimeDistributed, Activation, SimpleRNN, GRU
from keras.layers.core import Flatten, Dense, Dropout, Lambda
from keras.optimizers import SGD, RMSprop, Adam
from keras.utils.data_utils import get_file
from numpy.random import random, permutation, randn, normal, uniform, choice

Using TensorFlow backend.


Let's start by getting the text from Project Guttenberg at the link below and strip the header and footer.

The whole text is converted to lowercase to reduce the overall dictionary length. It would be more interesting to keep the upper case characters and see if the model can correctly employ them. We also convert vowels with umlaut (ä, ë, ...) to unaccented vowels, for the same reason.

In [29]:
PATH='/Users/marcototolo/Projects/dantegen/'

In [32]:
fpath = get_file(PATH + 'data/raw/divcomm.txt', origin="http://www.gutenberg.org/files/1012/1012-0.txt")
text = open(fpath,encoding='utf8').read().lower()
text = text[932:-19658]
#umlaut = {'ä':'a','ë':'e','ï':'i','ö':'o','ü':'u','-':'—'}
#for word, initial in umlaut.items():
#    text = text.replace(word, initial)


Let's print the beginning and end of the corpus, as well as the total length.

In [33]:
print(text[:500])

la divina commedia
  di dante alighieri





  inferno




  inferno • canto i


  nel mezzo del cammin di nostra vita
  mi ritrovai per una selva oscura,
  ché la diritta via era smarrita.

  ahi quanto a dir qual era è cosa dura
  esta selva selvaggia e aspra e forte
  che nel pensier rinova la paura!

  tant’ è amara che poco è più morte;
  ma per trattar del ben ch’i’ vi trovai,
  dirò de l’altre cose ch’i’ v’ho scorte.

  io non so ben ridir com’ i’ v’intrai,
  tant’ era pien di sonno a que


In [34]:
print(text[-500:])

mètra che tutto s’affige
  per misurar lo cerchio, e non ritrova,
  pensando, quel principio ond’ elli indige,

  tal era io a quella vista nova:
  veder voleva come si convenne
  l’imago al cerchio e come vi s’indova;

  ma non eran da ciò le proprie penne:
  se non che la mia mente fu percossa
  da un fulgore in che sua voglia venne.

  a l’alta fantasia qui mancò possa;
  ma già volgeva il mio disio e ’l velle,
  sì come rota ch’igualmente è mossa,

  l’amor che move il sole e l’altre stelle.


In [35]:
len(text)

561094

Let's now get the set of all characters in the text and print them out. We'll add a null carachter for padding.

In [36]:
chars = sorted(list(set(text)))
chars.insert(0, "\0")
vocab_size = len(chars)

In [37]:
vocab_size

56

In [38]:
"".join(chars)

'\x00\n !(),-.:;?abcdefghijlmnopqrstuvxyz«»àäèéëìïòóöùü—‘’“”•'

Now we have to assign an index to each character.

In [39]:
char_indices = {c: i for i, c in enumerate(chars)}
indices_char = {i: c for i, c in enumerate(chars)}

In [40]:
idx = [char_indices[c] for c in text]

Now idx contains the whole divine comedy text encoded with the indeces we've just created.

In [41]:
idx[:18] # = "LA DIVINA COMMEDIA"

[22, 12, 2, 15, 20, 32, 20, 24, 12, 2, 14, 25, 23, 23, 16, 15, 20, 12]

In [42]:
print(''.join(indices_char[i] for i in idx[:18]))

la divina commedia


# Creating the model

Our model will take the first n characters and try to predict the next one. Let's take n=40.

We'll create all the 40 chars sequences in the text (sentences) and associate them to the 1-char shifted corresponding sequence (next_chars).

In [43]:
maxlen = 40
sentences = []
next_chars = []
for i in range(0, len(idx) - maxlen+1):
    sentences.append(idx[i: i + maxlen])
    next_chars.append(idx[i+1: i+maxlen+1])
print('nb sequences:', len(sentences))

nb sequences: 561055


Now we cast them in np arrays and throw away the very last one.

In [44]:
sentences = np.concatenate([[np.array(o)] for o in sentences[:-2]])
next_chars = np.concatenate([[np.array(o)] for o in next_chars[:-2]])

In [45]:
sentences.shape, next_chars.shape

((561053, 40), (561053, 40))

In [46]:
n_fac = 24 #number of embeddings
batch_size = 64
LSTM_units = 512

Next we define our model: we start with an embedding layer, followed by two LSTM networks and a fully connected layer with softmax to obtain each character probability.

In [47]:
model=Sequential([
        Embedding(vocab_size, n_fac, input_length=maxlen),
        LSTM(LSTM_units, return_sequences=True, dropout=0.2, recurrent_dropout=0.2, 
             implementation=2),
        Dropout(0.2),
        LSTM(LSTM_units, return_sequences=True, dropout=0.2, recurrent_dropout=0.2, 
             implementation=2),
        Dropout(0.2),
        TimeDistributed(Dense(vocab_size)),
        Activation('softmax')
    ])

In [48]:
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam())

In [49]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 40, 24)            1344      
_________________________________________________________________
lstm_3 (LSTM)                (None, 40, 512)           1099776   
_________________________________________________________________
dropout_3 (Dropout)          (None, 40, 512)           0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 40, 512)           2099200   
_________________________________________________________________
dropout_4 (Dropout)          (None, 40, 512)           0         
_________________________________________________________________
time_distributed_2 (TimeDist (None, 40, 56)            28728     
_________________________________________________________________
activation_2 (Activation)    (None, 40, 56)            0         
Total para

# Train the model

Here's a function to print an example of text generated from the model.

In [50]:
def print_example(n=500, seed_string="nel mezzo del cammin di nostra vita\n  mi"):
    for i in range(n):
        x=np.array([char_indices[c] for c in seed_string[-maxlen:]])[np.newaxis,:]
        preds = model.predict(x, verbose=0)[0][-1]
        preds = preds/np.sum(preds)
        next_char = choice(chars, p=preds)
        seed_string = seed_string + next_char
    print(seed_string)

Now we start training the model.

In [105]:
model.fit(sentences, np.expand_dims(next_chars,-1), batch_size=64, epochs=3)

Epoch 1/2
561053/561053 [==============================] - 14808s - loss: 1.5459 
Epoch 2/2
561053/561053 [==============================] - 14826s - loss: 1.3143 


In [144]:
model.save_weights('dante1.h5')

In [51]:
model.load_weights('../models/dantegen_01_keras_cpu.h5')

In [52]:
print_example()

nel mezzo del cammin di nostra vita
  mi scende, né color non tenea far di quel giù battesmo.

  «o al fangosto del lagrome femmine,

  di qua tanto pur che li bassi levi.

  e prima vorrà che tu riguardi
  perché ’l tempo sono era e quel ch’i’ odo,
  dove più mortali avea giovanni e ville,

  sopra risposta il gelo a la virtù s’infrai.

  di quel ch’è nel vero a questa spene,
  se tu sorrisi e dinanzi tenea tutto ’l fine in su la poppa,
  loda di grasde arco d’ubidiso ecquisto,
  sì come abression là nostra volta qua degna;
  ma ques


This might now make any sense unless you know some Italian, but you can see that the model correctly uses punctuation like full stops and question marks. 

In [115]:
model.optimizer.lr=0.001

In [116]:
model.fit(sentences, np.expand_dims(next_chars,-1), batch_size=64, epochs=1)

Epoch 1/1
561053/561053 [==============================] - 14833s - loss: 1.2712 


In [117]:
print_example()

nel mezzo del cammin di nostra vita
  mi facea le prime la suora d’amore,
  onde per mostrarsi quïetato il sol percuote.

  e vidi le labbra dentro a quel di grazia s’aia;
  sola gente quale in paura si rincopra».

  poi comincia’ io; ed elli a me: «l’anime salete?

  com’ a la parola a la luce di gran sembiante,
  montavano a la muffa e di là ond’ io scuso;
  ma più el la faccia l’argomentar chiisa.

  poscia vid’ io parea trïunfar lo martar del tuo dimando».

  e io: «colui che mi parea tanto sì fatto,

  e poi che son sterpi e tu d


In [118]:
model.optimizer.lr=0.0001

In [119]:
model.fit(sentences, np.expand_dims(next_chars,-1), batch_size=64, epochs=1)

Epoch 1/1
561053/561053 [==============================] - 14884s - loss: 1.2482 


In [121]:
print_example()

nel mezzo del cammin di nostra vita
  mi fu correva la fronde piani,
  ritrarrote a la parte onde si volgieno,
  per viva unvider veder con porpo
  scendere un poco a la tua conoscescia,
  tratto ne l’aere antico e torto,
  pur giù nel dir diedi ’l monte, da li alberti
  voce se ne faran presso e margarita,
  grida fortuna è colei che la ingama bolgia
  si mosse, e falsai nel loco ove circunsto.

  vinser li ambo le timi, e con saranno troia,
  tanto dismontando, fu creato,
  da estre, dopo la gente fansi,
  e ora, e crescia i segni d


In [122]:
model.optimizer.lr=0.00001

In [123]:
model.fit(sentences, np.expand_dims(next_chars,-1), batch_size=64, epochs=3)

Epoch 1/3
561053/561053 [==============================] - 19911s - loss: 1.2316 
Epoch 2/3
561053/561053 [==============================] - 14819s - loss: 1.2179 
Epoch 3/3
561053/561053 [==============================] - 15258s - loss: 1.2060 


In [124]:
print_example()

nel mezzo del cammin di nostra vita
  mi rimosse; e tutti suoi convene
  fin questi verso le braccia aperse,
  trovai d’altro secondo quento schiera,
  poi disse: «ché noi a cui novelle sazio».

  queste parole prieghe mar ch’a le mie porte
  tutte raggia fiammelle, a veder ch’a bene star mie
  quando si parrà di beccheria di mangia
  dove pur l’ovra non sarebbe non si sponda.

  or vien quella che merta richiude,
  e non principio a chi lo sveglio in alcun sabo.

  quand’ io avea poi d’una sete onde si ricorda,

  come vel che la sen


In [125]:
model.save_weights('dante1.h5')

In [128]:
print_example()

nel mezzo del cammin di nostra vita
  mi serva pria parlere, ad amar per nave,
  solo a tante malizia fosser di piacenti,
  prendere a veder, ché pur idïente acro
  lo bulicame al mar si fu ammontibero.

  lume ch’avrebber colto da le gran cima

  di riguardar la barba in richiusa la creatura,
  da l’armi», disse, «che siete voi?»,
  diss’ io, «in sù hanno di là sù nel mondo,
  lagrimando al fine ove si può sembiante;
  e però l’atto d’amor m’era più vera,

  anzi si stavan tanto spiriti giude,
  disse: «come ii vidi già nel remo è sì


In [135]:
print_example(1000)

nel mezzo del cammin di nostra vita
  mi vide così si stozzasse
  per tante stelle membra che la maria
  sovr’ a cuuro e caldi la testa pianta,
  prender le facelle del roman la coglia
  che l’ali nasconde e corpo tutto fossi;

  ed el sensibil cosa o dannata;
  volsersi a noi poscia che ristetter li miei,
  a bene arviver, facea male.

  le membra più e più e per tre mlii,
  odor che l’altro tanto non si scossa.

  «o dolce poi di lor sapienza in sue figure scale;
  avviso ancor non lasciava icinto e tarda,
  ch’ardente da lucca a la sua mano a tutte quante.

  e come a quel che sia la natura
  che da lui le membra con le tue amor non rechera.

  ombra v’eran colà, con pomi si siicchia.

  lo vostro mondo sovra questo volto,
  come ’l cerchio del cammin ci si specchia,
  a dio verace di ciascuna reina,
  che non soccorre assai è più lungo speranza,
  come tempo ha del suo riguardar di questo,
  dispuos’ io lui, «gridò: «fronda,
  poro i piedi in lor felice,
  poscia ch’elli ebbe chius

After training for a few more epochs, the results are interesting in terms of language used, but we still see a few mistakes: direct speech chunks are sometimes opened and not closed; the three line (tercet) pattern is mostly not used and the rhyming also doesn't work.

Let's try a few combination of the model parameters and see if we can get some better results.

In [171]:
MAX_LEN = [40,60]
EMB = [24,32]
LSTM_UNITS = [512,1024]

In [ ]:
def get_sequences(maxlen):
    sentences = []
    next_chars = []
    for i in range(0, len(idx) - maxlen+1):
        sentences.append(idx[i: i + maxlen])
        next_chars.append(idx[i+1: i+maxlen+1])
    sentences = np.concatenate([[np.array(o)] for o in sentences[:-2]])
    next_chars = np.concatenate([[np.array(o)] for o in next_chars[:-2]])
    return (sentences, next_chars)

def get_model(maxlen, embeddings, LSTM_units):
    model=Sequential([
        Embedding(vocab_size, embeddings, input_length=maxlen),
        LSTM(LSTM_units, return_sequences=True, dropout=0.2, recurrent_dropout=0.2, 
             implementation=2),
        Dropout(0.2),
        LSTM(LSTM_units, return_sequences=True, dropout=0.2, recurrent_dropout=0.2, 
             implementation=2),
        Dropout(0.2),
        TimeDistributed(Dense(vocab_size)),
        Activation('softmax')
    ])
    model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam())
    return model

In [ ]:
from keras.callbacks import History
k = 1
losses = []
for maxlen in MAX_LEN:
    sentences, next_chars = get_sequences(maxlen)
    for embeddings in EMB:
        for units in LSTM_UNITS:
            k = k + 1
            model = get_model(maxlen, embeddings, units)
            model.optimizer.lr = 0.01
            history = History()
            loss = 3
            i = 0
            while (loss>1.3) & (i<6):
                i = i + 1
                model.fit(sentences, np.expand_dims(next_chars,-1), batch_size=64, epochs=1, 
                     callbacks=[history])
                loss = history.history['loss'][-1]
            print_example()
            for j in [0.001,0.0001,0.00001,0.00001,0.00001]
                model.optimizer.lr = j
                model.fit(sentences, np.expand_dims(next_chars,-1), batch_size=64, epochs=1, 
                     callbacks=[history])
                print_example()
            losses.append(history.history['loss'][-1])
            model.save_weights('dante' + str(k))